<a href="https://colab.research.google.com/github/pikanaeri/plm-model-comparison/blob/main/efam-performance/EFAM_VPF_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EFAM VPF prediction

requires efam sequence:embedding dictionary
to load the model, make sure that Tensorflow is updated to 2.16 or above


In [ ]:
sudo apt-get update -qq
sudo apt-get install -y tmux
tmux

In [ ]:
sudo apt-get update
sudo apt-get install bzip2 libxml2-dev
sudo apt upgrade
sudo apt-get install wget
wget https://repo.anaconda.com/archive/Anaconda3-2022.05-Linux-x86_64.sh
bash Anaconda3-2022.05-Linux-x86_64.sh
rm Anaconda3-2022.05-Linux-x86_64.sh

In [ ]:
sudo fallocate -l 15G /swapfile
sudo chmod 600 /swapfile
ls -lh /swapfile
sudo mkswap /swapfile
sudo swapon /swapfile

#@markdown Append "/swapfile swap swap defaults 0 0" to the /etc/fstab file using a text editor of your choice
sudo nano /etc/fstab

/swapfile swap swap defaults 0 0

In [ ]:
sudo apt-get install git
sudo apt install python3-pip

python3 --version
pip3 --version

In [ ]:
pip install transformers
pip3 install torch torchvision torchaudio transformers sentencepiece accelerate --extra-index-url https://download.pytorch.org/whl/cu116
pip install tensorflow
python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 17.9 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
import pickle
from ast import literal_eval
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from tqdm import tqdm


phrog_palette = {
    'DNA, RNA and nucleotide metabolism': 'red',
    'connector': 'blue',
    'head and packaging': 'green',
    'integration and excision': 'pink',
    'lysis': 'gray',
    'moron, auxiliary metabolic gene and host takeover': 'brown',
    'other': 'purple',
    'tail': 'darkorange',
    'transcription regulation': 'cyan',
    'unknown': 'black'
}

#load efam super annotations file
efam_super_annote = pd.read_csv('Final_Super_Condensed_Annotations-updated_efam.tsv', sep='\t')
efam_super_annote['Proteins'] = efam_super_annote['Proteins'].apply(literal_eval)


#load trained model
model = tf.keras.models.load_model('models/model_unknown.keras')
classes = pickle.load(open('models/model_unknown_lb.pkl', 'rb'))


#load necessary dictionaries for efam identifer and cluster lookups
ci = {}
for c in tqdm(efam_super_annote['Cluster']):
    ps = efam_super_annote[efam_super_annote['Cluster'] == c]['Proteins'].item()
    ci[c] = ps

iv = pickle.load(open('identifier_to_vector_protbert_bdf_08132024.pkl', 'rb')) #file containing the dictionary mapping EFAM name to embedding vector

cs = list(ci.keys())

cs_preds = []
cs_prediction_per_class = []
for c in tqdm(cs):
    vs = [iv[i] for i in ci[c] if i in iv.keys()]
    if len(vs) < len(ci[c]):
        print(c)
        if len(vs) < 1:
            continue
    vs = np.vstack(vs)
    pred = model.predict(vs, verbose=0)
    pred_mean = np.mean(pred, axis=0)
    pred_sd = np.std(pred, axis=0)
    pred_score = pred_mean[pred_mean.argmax()]
    pred_sd = pred_sd[pred_mean.argmax()]
    pred_c = classes.classes_[pred_mean.argmax()]
    cs_preds.append((c, pred_c, pred_score, pred_sd, len(vs)))
    cs_prediction_per_class.append((
        c,
        pred_mean[0],
        pred_mean[1],
        pred_mean[2],
        pred_mean[3],
        pred_mean[4],
        pred_mean[5],
        pred_mean[6],
        pred_mean[7],
        pred_mean[8],
        pred_mean[9]
    ))


cs_assignment = pd.DataFrame(cs_preds, columns=['cluster', 'label', 'average_prob', 'sd_prob', 'number_of_sequences'])
cs_assignment.to_csv('efam_clusters_predictions.tsv', sep='\t', index=False)

cols = ('cluster',
        classes.classes_[0],
        classes.classes_[1],
        classes.classes_[2],
        classes.classes_[3],
        classes.classes_[4],
        classes.classes_[5],
        classes.classes_[6],
        classes.classes_[7],
        classes.classes_[8],
        classes.classes_[9]
       )
cs_assignment_1 = pd.DataFrame(cs_prediction_per_class, columns=cols)
cs_assignment_1.to_csv('efam_clusters_all_probabilities.tsv', sep='\t', index=False)


